In [ ]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
from datetime import datetime
import math
np.set_printoptions(precision=4)
import sys; sys.path.append('book_scripts')
%pwd

In [ ]:
# input Base CSV数据文件
input_base_index_data_csv = 'E:/project/pychram/traderesp/base/output-csv/output-sh000001-2013-2014-index.csv'
input_base_stock_data_csv = 'E:/project/pychram/traderesp/base/output-csv/output-2013-2014-day-stock-history.csv'

In [ ]:
# input strategy CSV数据文件
input_strategy_stock_data_csv = 'E:/project/pychram/traderesp/strategy/output-csv/output-stock-2014-5day-increase.csv'

In [ ]:
# 注意：这里请填写数据文件在您电脑中的路径，并注意路径中斜杠的方向
base_index_data = pd.read_csv(input_base_index_data_csv,parse_dates=['date'],encoding='gbk')

In [ ]:
base_stock_data = pd.read_csv(input_base_stock_data_csv,parse_dates=['date'],encoding='gbk')

In [ ]:
strategy_stock_data = pd.read_csv(input_strategy_stock_data_csv,parse_dates=['date'],encoding='gbk')

In [ ]:
base_index_data.columns

In [ ]:
base_stock_data.columns

In [ ]:
strategy_stock_data.columns

In [ ]:
max_use_money = 0.00
cur_use_money = 0.00
cur_stock_list = []
stock_buy_sell_history_list = []

In [ ]:
def check_and_add_money():
    global max_use_money
    global cur_use_money
    if cur_use_money < 100000.00 :
        max_use_money += 50000.00
        cur_use_money += 50000.00

In [ ]:
def check_stock_list_empty():
    global cur_stock_list
    if len(cur_stock_list) > 0:
        return False
    else:
        return True

In [ ]:
def show_cur_msg():
    print "max_use_money =",max_use_money
    print "cur_use_money =",cur_use_money
    print "cur_stock_list count() =",len(cur_stock_list)
    print "stock_buy_sell_history_list count() =",len(stock_buy_sell_history_list)

In [ ]:
def do_buy_stock(cur_date,strategy_stock):
    global base_stock_data
    global cur_use_money
    global cur_stock_list
    # print strategy_stock
    for id, row in strategy_stock.iterrows():
        #print id,row
        # 1. check money num
        check_and_add_money()
        # 2. 
        code = row['code'].strip('\'')
        tmp_df = base_stock_data[(base_stock_data['date'] == cur_date) & (base_stock_data['code'] == int(code))].dropna()
        #print len(tmp_df)
        if len(tmp_df) > 0:
            #print tmp_df['close']
            #type(tmp_df['close'])
            buy_price = float(tmp_df['back_close'])
            buy_hand_num = math.ceil(10000/(buy_price*100))
            tmp_buy_rec = {'code':code,
                           'buy_price':buy_price,
                           'buy_num':buy_hand_num*100,
                           'buy_date':cur_date,
                           'buy_money':buy_hand_num*100*buy_price}
            cur_use_money -= buy_hand_num*100*buy_price
            cur_stock_list.append(tmp_buy_rec)    

In [ ]:
def do_sell_stock(cur_date):
    global base_stock_data
    global cur_use_money
    global cur_stock_list
    global stock_buy_sell_history_list
    
    # 当前没有可以卖出的股票
    if len(cur_stock_list) == 0 :
        return False
    # 遍历列表
    need_delete_list = []
    for item in cur_stock_list:
        code = item['code']
        buy_price =item['buy_price']
        buy_num = item['buy_num']
        buy_money = item['buy_money']
        buy_date = item['buy_date']
        # 查询 当天的股价
        tmp_df = base_stock_data[(base_stock_data['date'] == cur_date) & (base_stock_data['code'] == int(code))].dropna()
        # 当天没有该股票信息,跳过
        if len(tmp_df) == 0:
            #print '没有找到该股票信息'
            continue
        # 
        sell_price = float(tmp_df['back_close'])
        sell_num = buy_num
        sell_money = sell_price * sell_num
        sell_date = cur_date
        #
        need_delete_list.append(item)
        #
        old_item = item
        old_item['sell_price'] = sell_price
        old_item['sell_num'] = sell_num
        old_item['sell_money'] = sell_money
        old_item['sell_date'] = sell_date
        stock_buy_sell_history_list.append(old_item)
        #
        cur_use_money += sell_money
    
    #删除
    for item in need_delete_list:
        cur_stock_list.remove(item)

In [ ]:
def do_sell_condition_stock(cur_date):
    global base_stock_data
    global cur_use_money
    global cur_stock_list
    global stock_buy_sell_history_list
    
    # 当前没有可以卖出的股票
    if len(cur_stock_list) == 0 :
        return False
    # 遍历列表
    need_delete_list = []
    for item in cur_stock_list:
        code = item['code']
        buy_price =item['buy_price']
        buy_num = item['buy_num']
        buy_money = item['buy_money']
        buy_date = item['buy_date']
        # 0. 当天的不计算
        if buy_date == cur_date:
            #print '当天买的'
            continue
            
        sell_price = 0.00    
        # 0. 超过 3 天，强制卖出
        if (cur_date - buy_date).days >= 8:
            # 查询当天的股价
            tmp_df = base_stock_data[(base_stock_data['date'] == cur_date) 
                                     & (base_stock_data['code'] == int(code))].dropna()
            # 当天没有该股票信息,跳过
            if len(tmp_df) == 0:
                #print '没有找到该股票信息'
                continue
            else:
                sell_price = float(tmp_df['back_close'])
        # 1. 3 天内，判断是否死叉    
        else:
            # 1. 查询当天的卖出条件
            # 2. 符合卖出条件，返回当天的股价
            tmp_df = base_stock_data[(base_stock_data['date'] == cur_date) 
                                     & (base_stock_data['code'] == int(code))
                                     & (base_stock_data['today_kdj_x_signal'] == 1)].dropna()
            # 2.1 没有死叉,跳过
            if len(tmp_df) == 0:
                #print '没有找到该股票信息'
                continue
            else:
                sell_price = float(tmp_df['back_close'])
        
        #
        sell_num = buy_num
        sell_money = sell_price * sell_num
        sell_date = cur_date
        #
        need_delete_list.append(item)
        #
        old_item = item
        old_item['sell_price'] = sell_price
        old_item['sell_num'] = sell_num
        old_item['sell_money'] = sell_money
        old_item['sell_date'] = sell_date
        stock_buy_sell_history_list.append(old_item)
        #
        cur_use_money += sell_money
    
    #删除
    for item in need_delete_list:
        cur_stock_list.remove(item)

In [ ]:
def run_strategy():
    global base_index_data
    global strategy_stock_data

    # 遍历
    for id,row in base_index_data.iterrows():
        #空值跳过
        if row.isnull().any():
            continue

        # 当前时间加一天的时间区间，仍然为当天
        cur_date = row['date']
        #print cur_date

        # 1.可买状态
        if int(row['today_signal']) == 1 :
            #print row
            #print int(row['today_signal'])
            # 1. check stock list
            # 判断当天是否有策略
            cur_day_strategy_stock = strategy_stock_data[strategy_stock_data['date'] == cur_date].dropna()
            #print cur_day_strategy_stock
            # 1.1 当天没有 stragecy stock list，不需处理,
            if len(cur_day_strategy_stock) == 0 :
                #print cur_date,'当天没有策略'
                pass
            # 1.2 当天有 stragecy stock list,买入 stock_list   
            else:
                do_buy_stock(cur_date,cur_day_strategy_stock)
                #break
            # 2. check cur_stock_list，看是否有符合卖出条件的 stock
            do_sell_condition_stock(cur_date)
        # 2. 可卖状态        
        if int(row['today_signal']) == 0 :
            #print row
            #print int(row['today_signal'])
            # 1. 判断当前 stock 列表是否为空
            is_empty = check_stock_list_empty()
            # 1.1 当前 stock 列表为空，不需处理,跳过
            if is_empty:
                #print cur_date,'当天 Stock 列表为空'
                continue
            # 1.2 当前有 stock list    
            else:
                do_sell_stock(cur_date)
                #break;        

In [ ]:
# 统计
def gen_staticstics():
    global base_index_data
    global cur_stock_list
    end_date = base_index_data['date'].max()
    do_sell_stock(end_date)
    # 生成可统计 dataframe
    # 将仍然没有卖出的，记为最后一日卖出,价格等为买入价格
    new_list = []
    for item in cur_stock_list:
        old_item = item
        old_item['sell_price'] = item['buy_price']
        old_item['sell_num'] = item['buy_num']
        old_item['sell_money'] = item['buy_money']
        old_item['sell_date'] = end_date
        new_list.append(old_item)
    #
    stat1 = pd.DataFrame(stock_buy_sell_history_list)
    stat2 = pd.DataFrame(new_list)
    stat = pd.concat([stat1,stat2],ignore_index=True) 
    show_staticstics(stat)
    return stat

In [ ]:
def show_staticstics(stat):
    global max_use_money 
    global stock_buy_sell_history_list 
    #
    action_num = len(stock_buy_sell_history_list)
    stat_sum_all = stat.sum()
    code_unique_num = len(stat['code'].unique())
    s_ge_b_num = stat[stat['sell_price'] > stat['buy_price']].count()
    #
    print '最大使用资金额 :',max_use_money 
    print '买入股票总数(排重) :',code_unique_num
    print '买卖总次数(一次买卖算一次) :',action_num
    print '其中获利次数 :',s_ge_b_num['code']
    print '其中亏损次数 :',(action_num - s_ge_b_num['code'])
    print '获利次数比率 :',s_ge_b_num['code'] / action_num
    print '亏损次数比率 :',(action_num - s_ge_b_num['code']) / action_num
    print '买盘资金累计 :',stat_sum_all['buy_money']
    print '卖盘资金累计 :',stat_sum_all['sell_money']
    print '利润累计 :',(stat_sum_all['sell_money'] - stat_sum_all['buy_money'])
    print '利润率 :',(stat_sum_all['sell_money'] - stat_sum_all['buy_money']) / max_use_money
    
    tmp = stat[stat['sell_price'] > stat['buy_price']].sum()
    print '获利股票利润率 :',(tmp['sell_money'] - tmp['buy_money']) /  max_use_money
    tmp = stat[stat['sell_price'] <= stat['buy_price']].sum()
    print '亏损股票利润率 :',(tmp['sell_money'] - tmp['buy_money']) /  max_use_money
    
    
    

In [ ]:
run_strategy()

In [ ]:
stat = gen_staticstics()